# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop). Если нет аккаунта на кеггле, скачать датасет можно [здесь](https://drive.google.com/file/d/1rLSr49zx6RPZIn7PV_LQr9KnnpPhrr0K/view?usp=sharing).

https://github.com/hse-mlds/ml/blob/main/base_group/lesson_18/MLHS_Recsys_1.ipynb

https://github.com/hse-mlds/ml/blob/main/base_group/lesson_18/MLHS_recsys_2.ipynb

# Загрузка и предобработка данных

In [1]:
import math

import numpy as np
import pandas as pd

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
!gdown --id 1932O9d4D1_ohjeV-VivI2oucVQCihK0L
!mkdir data
!unzip ./DeskDrop-Articles.zip -d data
!rm DeskDrop-Articles.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1932O9d4D1_ohjeV-VivI2oucVQCihK0L
To: /content/DeskDrop-Articles.zip
100% 8.59M/8.59M [00:00<00:00, 53.4MB/s]
Archive:  ./DeskDrop-Articles.zip
  inflating: data/shared_articles.csv  
  inflating: data/users_interactions.csv  


In [3]:
PATH_DATA = "./data"

In [4]:
articles_df = pd.read_csv(f"{PATH_DATA}/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [5]:
interactions_df = pd.read_csv(f"{PATH_DATA}/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [6]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [7]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(
    lambda x: event_type_strength[x]
)

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [8]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [9]:
interactions_from_selected_users_df = interactions_df.loc[
    np.in1d(interactions_df.personId, users_with_enough_interactions_df)
]

In [10]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [11]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [12]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [13]:
interactions = (
    interactions_train_df.groupby("personId")["contentId"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = interactions_test_df.groupby("personId")["contentId"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    ""
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


__Baseline__

Модель которая рекомендует наиболее популярные статьи. По условию будем рекомендовать 10 статей.

Определим наиболее популярные статьи по сумме оценок взаимодейтсвия с ними.

In [14]:
popular_content = (
    interactions_train_df.groupby("contentId")
    .eventStrength.sum()
    .reset_index()
    .sort_values("eventStrength", ascending=False)
    .contentId.values
)

popular_content

array(['-6783772548752091658', '-133139342397538859',
       '-8208801367848627943', ..., '6240076106289531207',
       '6541551984368289722', '7083316110921342538'], dtype=object)

In [15]:
print(articles_df.loc[articles_df.contentId == popular_content[1]]["title"].values)

['Novo workaholic trabalha, pratica esportes e tem tempo para a família. Conheça']


Теперь порекомендуем пользователям самые попоулярные статьи, такие с которыми  они еще не взаимодействовали.

In [16]:
top_k = 10

interactions["prediction_popular"] = interactions["true_train"].apply(
    lambda x: popular_content[~np.in1d(popular_content, x)][:top_k]
)

interactions["prediction_popular"][0]

array(['-6783772548752091658', '-133139342397538859',
       '-8208801367848627943', '8224860111193157980',
       '7507067965574797372', '-2358756719610361882',
       '-6843047699859121724', '-1297580205670251233',
       '8657408509986329668', '3367026768872537336'], dtype=object)

In [17]:
interactions.head(3)

,true_train,true_test,prediction_popular
personId,,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72...","[-6783772548752091658, -133139342397538859, -8..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -...","[-6783772548752091658, -133139342397538859, -8..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -...","[-6783772548752091658, -8208801367848627943, 8..."


Посчитаем для каждого пользователя долю угаданных рекомендаций. Усредним по всем пользователям.

In [18]:
def calc_precision(column):
    return (
        interactions.apply(
            lambda row: len(set(row["true_test"]).intersection(set(row[column])))
            / min(len(row["true_test"]) + 0.001, 10.0),
            axis=1,
        )
    ).mean()

In [19]:
calc_precision("prediction_popular")

0.006454207722621089

# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [20]:
!pip install lightfm > None

In [121]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (1.5 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [22]:
from scipy.sparse import csr_matrix

In [23]:
def sparse(users, contents, df):
    # Создаем словари для быстрого доступа к индексам
    user_to_idx = {user: idx for idx, user in enumerate(users)}
    content_to_idx = {content: idx for idx, content in enumerate(contents)}

    # Инициализируем разреженную матрицу
    data_matrix = csr_matrix((len(users), len(contents)), dtype=np.float32)

    # Заполняем матрицу данными из DataFrame
    for idx, row in df.iterrows():
        user_idx = user_to_idx.get(row["personId"])
        content_idx = content_to_idx.get(row["contentId"])
        if pd.notnull(row["eventStrength"]):
            data_matrix[user_idx, content_idx] = row["eventStrength"]

    return data_matrix

In [24]:
all_users = set(interactions_test_df["personId"]) | set(
    interactions_train_df["personId"]
)
all_contents = set(interactions_train_df["contentId"]) | set(
    interactions_test_df["contentId"]
)

In [25]:
data_train = sparse(all_users, all_contents, interactions_train_df)
data_test = sparse(all_users, all_contents, interactions_test_df)

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


## Задание 2 (0.5 балла)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [26]:
model = LightFM(k=10, loss="warp", random_state=0)
model.fit(data_train, epochs=50, num_threads=2)

print("precision@10:", precision_at_k(model, data_test, data_train, k=10).mean())

precision@10: 0.0059063137


## Задание 3 (2 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
tfidf_vectorizer = TfidfVectorizer()
item_features_matrix = tfidf_vectorizer.fit_transform(articles_df["text"])

In [29]:
model = LightFM(k=10, loss="warp")
model.fit(data_train, item_features=item_features_matrix, epochs=50, num_threads=2)

In [30]:
precision = precision_at_k(
    model, data_test, data_train, item_features=item_features_matrix, k=10
).mean()
print("precision@10:", precision)

precision@10: 0.0073319753


## Задание 4 (1.5 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [31]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [32]:
# Загружаем стоп-слова
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Инициализируем лемматизатор и векторизатор TF-IDF
lemmatizer = WordNetLemmatizer()
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [33]:
# Функция для предобработки текста
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Токенизация
    words = word_tokenize(text)
    # Удаление стоп-слов и лемматизация
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

In [34]:
articles_df["text"].iloc[0]

'All of this work is still very early. The first full public version of the Ethereum software was recently released, and the system could face some of the same technical and legal problems that have tarnished Bitcoin. Many Bitcoin advocates say Ethereum will face more security problems than Bitcoin because of the greater complexity of the software. Thus far, Ethereum has faced much less testing, and many fewer attacks, than Bitcoin. The novel design of Ethereum may also invite intense scrutiny by authorities given that potentially fraudulent contracts, like the Ponzi schemes, can be written directly into the Ethereum system. But the sophisticated capabilities of the system have made it fascinating to some executives in corporate America. IBM said last year that it was experimenting with Ethereum as a way to control real world objects in the so-called Internet of things. Microsoft has been working on several projects that make it easier to use Ethereum on its computing cloud, Azure. "Et

In [35]:
# Предобработка текста статей
articles_df["preprocessed_text"] = articles_df["text"].apply(preprocess_text)

In [36]:
articles_df["preprocessed_text"].iloc[0]

"work still early . first full public version ethereum software recently released , system could face technical legal problem tarnished bitcoin . many bitcoin advocate say ethereum face security problem bitcoin greater complexity software . thus far , ethereum faced much le testing , many fewer attack , bitcoin . novel design ethereum may also invite intense scrutiny authority given potentially fraudulent contract , like ponzi scheme , written directly ethereum system . sophisticated capability system made fascinating executive corporate america . ibm said last year experimenting ethereum way control real world object so-called internet thing . microsoft working several project make easier use ethereum computing cloud , azure . `` ethereum general platform solve problem many industry using fairly elegant solution - elegant solution seen date , '' said marley gray , director business development strategy microsoft . mr. gray responsible microsoft 's work blockchains , database concept b

In [37]:
# Преобразуем предобработанный текст в TF-IDF матрицу признаков
tfidf_vectorizer = TfidfVectorizer()
item_features_matrix_preproc = tfidf_vectorizer.fit_transform(
    articles_df["preprocessed_text"]
)

In [38]:
# Создаем и обучаем модель LightFM с использованием TF-IDF признаков
model = LightFM(k=10, loss="warp")
model.fit(
    data_train, item_features=item_features_matrix_preproc, epochs=50, num_threads=2
)

precision = precision_at_k(
    model, data_test, data_train, item_features=item_features_matrix_preproc, k=10
).mean()
print("precision@10:", precision)

precision@10: 0.006415479


Улучшилось ли качество предсказания?

Качество модели с предварительной предобработкой текста оказалось сопоставимым с качеством модели до предобработки текста. Думаю что все-таки нужно разобраться с препроцессингом различных языков, попробовать стемминг слов.

## Задание 5 (1.5 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

https://stackoverflow.com/questions/49896816/how-do-i-optimize-the-hyperparameters-of-lightfm

In [39]:
import itertools
from time import time

In [40]:
def generate_hyperparameters():
    """
    Подбор гиперпараметров.
    """

    while True:
        yield {
            "no_components": np.random.randint(10, 100),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-6),
            "user_alpha": np.random.exponential(1e-6),
            "max_sampled": np.random.randint(5, 30),
            "num_epochs": np.random.randint(5, 30),
        }

In [41]:
def random_search(train_data, test_data, features, num_samples=10, num_threads=1):
    for hyperparams in itertools.islice(generate_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams, k=10, random_state=0)
        model.fit(
            train_data,
            epochs=num_epochs,
            num_threads=num_threads,
            item_features=features,
        )

        score = precision_at_k(
            model,
            test_data,
            train_data,
            item_features=features,
            k=10,
            num_threads=num_threads,
        ).mean()

        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)

In [42]:
%%time
(score, hyperparams, model) = max(
    random_search(data_train, data_test, item_features_matrix_preproc, num_threads=2),
    key=lambda x: x[0],
)
print("Лучший результат {} at {}".format(score, hyperparams))

Лучший результат 0.007230142597109079 at {'no_components': 81, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.19733034088225643, 'item_alpha': 3.701933204178711e-06, 'user_alpha': 1.7162298393966008e-06, 'max_sampled': 26, 'num_epochs': 26}
CPU times: user 1h 50min 40s, sys: 6.1 s, total: 1h 50min 46s
Wall time: 1h 2min 59s


In [ ]:
# Лучший результат 0.007230142597109079 at {'no_components': 81,
# 'learning_schedule': 'adadelta',
# 'loss': 'bpr',
# 'learning_rate': 0.19733034088225643,
# 'item_alpha': 3.701933204178711e-06,
# 'user_alpha': 1.7162298393966008e-06,
# 'max_sampled': 26,
# 'num_epochs': 26}
# CPU times: user 1h 50min 40s, sys: 6.1 s, total: 1h 50min 46s
# Wall time: 1h 2min 59s

In [112]:
model_best = LightFM(
    k=10,
    no_components=81,
    learning_schedule="adadelta",
    loss="bpr",
    learning_rate=0.19733034088225643,
    item_alpha=3.701933204178711e-06,
    user_alpha=1.7162298393966008e-06,
    max_sampled=26,
)

In [122]:
%%time
model_best.fit(
    data_train, item_features=item_features_matrix_preproc, epochs=26, num_threads=4
)

precision = precision_at_k(
    model_best, data_test, data_train, item_features=item_features_matrix_preproc, k=10
).mean()
print("precision@10:", precision)

precision@10: 0.007433809
CPU times: user 24min 42s, sys: 1.56 s, total: 24min 44s
Wall time: 15min 35s


Подбором гиперпараметров не удалось поднять качество модели. Необходимо опробовать каждый набор гиперпараметров из конечных последовательностей, например - по сетке.

## Задание 6 (1 балл)

Реализуйте функции для вычисления следующих метрик:
* precision@k
* recall@k
* NDCG@k



In [138]:
predictions = model_best.predict_rank(
    data_test, item_features=item_features_matrix_preproc, num_threads=4
)

In [139]:
usr_ind = 0
k_itms = 10

user_predictions = (
    predictions[usr_ind].toarray().flatten()
)  # Преобразование в массив и сглаживание
print("user_predictions:", user_predictions)

top_k_items = np.argsort(user_predictions)[::-1][:k_itms]
print("top_k_items:", top_k_items)

true_interactions = data_test.getrow(usr_ind).indices
print("true_interactions:", true_interactions)

k_itms = min(k_itms, len(true_interactions))  # Обновляем k_itms

correct_predictions = (
    len(set(top_k_items).intersection(set(true_interactions))) / k_itms
)

print(f"precision по одному пользователю с индексом {usr_ind}:", correct_predictions)

user_predictions: [0. 0. 0. ... 0. 0. 0.]
top_k_items: [1157  849 1353 2945 1037 2222   80 2954 2883  620]
true_interactions: [  37   80  235  277  298  459  527  620  833  849 1037 1100 1157 1215
 1268 1353 1542 1624 1647 1680 1715 1839 1849 2194 2212 2222 2387 2397
 2422 2787 2831 2883 2945 2954]
precision по одному пользователю с индексом 0: 1.0


Precision@k= Top-K релевантных элементов / K


In [125]:
def precision_at_k_imp(model, test_data, train_data, item_features, k=10):
    predictions = model.predict_rank(
        test_data, train_data, item_features=item_features, num_threads=1
    )

    precision = []

    # Проходим по всем пользователям
    for user_id in range(test_data.shape[0]):
        user_predictions = predictions[user_id]

        # Индексы объектов (items) с наибольшими предсказанными рейтингами
        top_k_items = np.argsort(user_predictions.toarray())[0][::-1][:k]

        # Истинные взаимодействия пользователя из тестовых данных
        true_interactions = set(test_data.getrow(user_id).indices)

        if true_interactions == 0:
            continue

        # Количество правильно предсказанных элементов в топе-K
        correct_predictions = len(
            set(top_k_items).intersection(true_interactions)
        ) / min(len(true_interactions) + 0.001, k)

        # # Добавляем точность для данного пользователя к общей точности
        precision.append(correct_predictions)

    # Вычисляем среднюю точность по всем пользователям
    precision = np.sum(precision) / test_data.shape[0]

    return precision

В данном месте у меня возникли затруднения:
- как обрабатывать случаи когда `true_interactions` меньше значения `k`;
- и как быть в том случае если `true_interactions` == 0;

От сюда вытекает проблема подсчета среднего. Среднее из всех наблюдений по `train` или только тех которые прошли обработку по условиям.

Решил не отсекать наблюдения у которых `k` меньше заданного, считая что у данного пользователя просто не достаточно данных для оценки точности, а учитывать и их. Т.е. среднее считать относительно всех наблюдений, даже тех, где количество `true_interactions` равно нулю.

Recall@k = Top-K релевантных элементов / Все релевантные элементы

In [126]:
def recall_at_k_imp(model, test_data, train_data, item_features, k=10):
    predictions = model.predict_rank(
        test_data, train_data, item_features=item_features, num_threads=1
    )

    recall = []

    for user_id in range(test_data.shape[0]):
        user_predictions = predictions[user_id]

        top_k_items = np.argsort(user_predictions.toarray())[0][::-1][:k]

        true_interactions = set(test_data.getrow(user_id).indices)

        relevant_items = len(true_interactions)

        if relevant_items == 0:
            continue

        correct_predictions = len(set(top_k_items).intersection(true_interactions))

        recall.append(correct_predictions / relevant_items)

    if not recall:
        return 0.0

    recall = np.sum(recall) / test_data.shape[0]

    return recall

In [127]:
def ndcg_at_k(model, test_data, train_data, item_features, k=10):
    predictions = model.predict_rank(
        test_data, train_data, item_features=item_features, num_threads=1
    )

    ndcg = []

    for user_id in range(test_data.shape[0]):
        user_predictions = predictions[user_id]

        top_k_items = np.argsort(user_predictions.toarray())[0][::-1][:k]

        true_interactions = set(test_data.getrow(user_id).indices)

        # Пропустим значения в которых k меньше заданного
        if len(true_interactions) < k:
            continue

        # Ранги рекомендованных элементов
        ranks = [
            (
                list(true_interactions).index(item_id) + 1
                if item_id in true_interactions
                else 0
            )
            for item_id in top_k_items
        ]

        # Discounted Cumulative Gain
        dcg = np.sum([1.0 / np.log2(rank + 2) for rank in ranks])

        # Порядок элементов для данного пользователя
        ideal_ranks = np.arange(len(true_interactions)) + 1

        # Ideal Discounted Cumulative Gain
        idcg = np.sum([1.0 / np.log2(rank + 2) for rank in ideal_ranks])

        # Вычисляем NDCG для данного пользователя и добавляем в список
        ndcg.append(dcg / idcg)

    # Вычисляем средний NDCG по всем пользователям
    ndcg = np.mean(ndcg)

    return ndcg

## Задание 7 (1 балл)

Вычислите значения реализованных метрик для $k=10$ для лучшей полученной модели в предыдущих шагах.

Найдите уже реализованные варианты этих метрик в библиотеках lightfm и sklearn. Сравните полученные у вас значения метрик с результатами встроенных в библиотеки метрик.

Оценим метрики на Test

In [128]:
precision_imp = precision_at_k_imp(
    model_best, data_test, data_train, item_features=item_features_matrix_preproc, k=10
)
recall_imp = recall_at_k_imp(
    model_best, data_test, data_train, item_features=item_features_matrix_preproc, k=10
)
ndcg_imp = ndcg_at_k(
    model_best, data_test, data_train, item_features=item_features_matrix_preproc, k=10
)

print("Ручная реализация метрик на test")
print("Precision_imp@10:", precision_imp)
print("Recall_imp@10:", recall_imp)
print("NDCG_imp@10:", ndcg_imp)

Ручная реализация метрик на test
Precision_imp@10: 0.8600866745660125
Recall_imp@10: 0.7572131442257417
NDCG_imp@10: 0.565920855869897


In [129]:
from lightfm.evaluation import precision_at_k, recall_at_k
from sklearn.metrics import ndcg_score

precision_lightfm = precision_at_k(
    model_best,
    data_test,
    train_interactions=data_train,
    item_features=item_features_matrix_preproc,
    k=10,
).mean()

recall_lightfm = recall_at_k(
    model_best,
    data_test,
    train_interactions=data_train,
    item_features=item_features_matrix_preproc,
    k=10,
).mean()

predict = model_best.predict_rank(
    data_test, data_train, item_features=item_features_matrix_preproc
)
data_test_dense = data_test.toarray().copy()
ndcg_lightfm = np.mean(ndcg_score(data_test_dense, predict.toarray(), k=10))

print("Библиотечная реализация метрик на test")
print("Precision@10:", precision_lightfm)
print("Recall@10:", recall_lightfm)
print("NDCG@10:", ndcg_lightfm)

Библиотечная реализация метрик на test
Precision@10: 0.007433809
Recall@10: 0.008381488176304715
NDCG@10: 0.7630191378985153


Оценим метрики на Train

In [130]:
precision_imp = precision_at_k_imp(
    model_best, data_train, None, item_features=item_features_matrix_preproc, k=10
)
recall_imp = recall_at_k_imp(
    model_best, data_train, None, item_features=item_features_matrix_preproc, k=10
)
ndcg_imp = ndcg_at_k(
    model_best, data_train, None, item_features=item_features_matrix_preproc, k=10
)

print("Ручная реализация метрик на train")
print("Precision_imp@10:", precision_imp)
print("Recall_imp@10:", recall_imp)
print("NDCG_imp@10:", ndcg_imp)

Ручная реализация метрик на train
Precision_imp@10: 0.885739205681781
Recall_imp@10: 0.6068588701183879
NDCG_imp@10: 0.4880700073767989


In [131]:
precision_lightfm = precision_at_k(
    model_best, data_train, item_features=item_features_matrix_preproc, k=10
).mean()
recall_lightfm = recall_at_k(
    model_best, data_train, item_features=item_features_matrix_preproc, k=10
).mean()

predict = model_best.predict_rank(
    data_train, item_features=item_features_matrix_preproc
)
data_train_dense = data_train.toarray().copy()
ndcg_lightfm = np.mean(ndcg_score(data_train_dense, predict.toarray(), k=10))

print("Библиотечная реализация метрик на train")
print("Precision@10:", precision_lightfm)
print("Recall@10:", recall_lightfm)
print("NDCG@10:", ndcg_lightfm)

Библиотечная реализация метрик на train
Precision@10: 0.5693345
Recall@10: 0.5416256219950102
NDCG@10: 0.708719061942834


Результаты сильно отличаются, в случае библиотечной реализации и ручной. Вероятнее всего ошибка в выборе метода `.predict_rank` и в том что не отсекаются наблюдения у которых `k` меньше заданного.

## Задание 8 (1 балл)

Реализуйте алгоритм ALS и примените его для решения задачи ноутбука.

**ALS**

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}$$

Суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя~$u$ и товара $i$, соответственно, матрицы $P, Q$ получаются путем записывания по столбцам векторов $p_u, q_i$ соответственно.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$

Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$

аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья c постановкой задачи для ALS на explicit feedback:**

* Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.

**Оригинальная статья с ALS для implicit данных, которая стала более известной:**

* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee.


In [132]:
class ALS:
    def __init__(self, num_factors=10, num_iterations=10, lambda_reg=0.01):
        self.num_factors = num_factors
        self.num_iterations = num_iterations
        self.lambda_reg = lambda_reg

    def fit(self, train_matrix):
        self.train_matrix = train_matrix
        self.num_users, self.num_items = train_matrix.shape

        self.train_matrix = self.train_matrix.tocsr()

        train_matrix_dense = self.train_matrix.toarray()

        self.P = np.random.rand(self.num_users, self.num_factors)
        self.Q = np.random.rand(self.num_items, self.num_factors)

        for _ in range(self.num_iterations):
            # Обновим P
            for u in range(self.num_users):
                self.P[u, :] = np.linalg.solve(
                    np.dot(self.Q.T, self.Q)
                    + self.lambda_reg * np.eye(self.num_factors),
                    np.dot(self.Q.T, train_matrix_dense[u, :].T),
                )
            # Обновим Q
            for i in range(self.num_items):
                self.Q[i, :] = np.linalg.solve(
                    np.dot(self.P.T, self.P)
                    + self.lambda_reg * np.eye(self.num_factors),
                    np.dot(self.P.T, train_matrix_dense[:, i]),
                )

    def predict(self, user_id, item_id):
        return np.dot(self.P[user_id], self.Q[item_id])

In [133]:
def get_predictions(model, test_matrix):
    num_users, num_items = test_matrix.shape
    predictions = np.zeros((num_users, num_items))
    for user_id in range(num_users):
        for item_id in range(num_items):
            predictions[user_id, item_id] = model.predict(user_id, item_id)
    return predictions


def precision_at_k(predictions, test_matrix, k=10):
    num_users = test_matrix.shape[0]
    precision_sum = 0
    for user_id in range(num_users):
        top_k_items = np.argsort(predictions[user_id])[::-1][:k]
        true_interactions = test_matrix[user_id].indices
        num_correct_predictions = len(
            set(top_k_items).intersection(set(true_interactions))
        )
        precision_sum += num_correct_predictions / k
    return precision_sum / num_users

In [134]:
als_model = ALS(num_factors=20, num_iterations=10, lambda_reg=0.1)

als_model.fit(data_train)

In [135]:
# Получаем предсказания для всех пользователей из data_test
predictions = get_predictions(als_model, data_test)

In [136]:
precision = precision_at_k(predictions, data_test)
print("Precision по k:", precision)

Precision по k: 0.0035964912280701767
